In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
table_name = dbutils.widgets.get("table")

In [0]:
def table_exists(catalog, schema, tablename):
    query = f"show tables from {catalog}.{schema}"
    table_ok = spark.sql(query).filter(f"tableName = '{tablename}'").count()

    return table_ok == 1

In [0]:
if not table_exists(catalog, schema, table_name):
    df = (
        spark.read.format("csv")
        .options(
            header=True,
            delimiter=",",
            inferSchema=True,
            encoding="latin1",
        )
        .load("/Volumes/raw-data/sistema_leitos/leitos/")
    )

    df = df.withColumnsRenamed({y: y.lower().replace(r" ", "_") for y in df.columns})

    (
        df.coalesce(1)
        .write.format("delta")
        .mode("overwrite")
        .saveAsTable(f"{catalog}.{schema}.{table_name}")
    )

# Como fazer a ingestão incremental em caso de CDC:
## 1 - Ler todos os arquivos do CDC
`spark.read.format("extensao").options(add_option).load("path_dos_arquivos")`
## 2 - Criar deduplicação baseado no id para pegar o arquivo mais recente
Ex: ordernar pela data de alteração e fazer um unique mantendo o primeiro registro
## 3 - Carregar a tabela delta atual
```
import delta

delta.DeltaTable.ForName(spark, "schema.nome_tabela")
```
## 4 - Fazer o UPSERT
merge da tabela atual com as novas informações designando o que fazer para cada tipo de operação
```
tab_atual.alias("ta")
.merge(tab_nova.alias("tn"), "ta.id = tn.id")
.whenMatchedDelete(condition = "tn.operation = 'D'")
.whenMatchedUpdateAll(condition = "tn.operation = 'U'")
.whenMatchedInsertAll(condition = "tn.operation = 'i'")
```

In [0]:
spark.catalog.tableExists("bronze.sistema_leitos.hospitais")